In [10]:
### DEEP LEARNING ###
from helper import *

## EXTRACTING X AND Y MATRICES. READ HELPER FOR FUNCTION INFO ##
dh = data_handle()
# EXTRACTING 50 SAMPLES OF CLASSES SPECIFIED IN INCLUDE FROM ALL IMGS
X, y = dh.featurize(resize=227, n=100, include=(2,5,6,7,8,9))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 51529 and the array at index 1 has size 206116

In [3]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# lb = preprocessing.LabelBinarizer()

# range_ = np.array(list(set(y)))
# out = lb.fit(range_)
# y_binary = out.transform(y.astype(int))

# from sklearn.preprocessing import StandardScaler
# scalify = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
# X_train = scalify.fit_transform(X_train)
# X_test = scalify.fit_transform(X_test)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X, y_binary, test_size=0.20, random_state=42)
# X_train_b = scalify.fit_transform(X_train_b)
# X_test_b = scalify.fit_transform(X_test_b)

In [4]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time

# a = np.empty((0,227,227,1))
# b = np.empty((0,227,227,1))
# for i in range(len(X_train)):
#     a = np.append(a, X_train[i].reshape(1,227,227,1), axis=0)
# for i in range(len(X_test)):
#     b = np.append(b, X_test[i].reshape(1,227,227,1), axis=0)

# train_ds = tf.data.Dataset.from_tensor_slices((a, y_train))
# test_ds = tf.data.Dataset.from_tensor_slices((b, y_test))
# train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
# test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()

# print(a.shape, b.shape)

In [5]:
alexnet = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation='softmax')
])

In [6]:
alexnet.compile(loss='binary_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
alexnet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        11712     
_________________________________________________________________
batch_normalization (BatchNo (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 384)       8

In [8]:
alexnet.fit(X_train.reshape(-1,227,227,1), y_train, epochs=30)

ValueError: cannot reshape array of size 6298560 into shape (227,227,1)